In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# default_exp decoder

In [10]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [11]:
#export
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting

import re

import string
import gc

from fastai2.basics import *
from fastai2.text.all import *
from fastai2.callback.all import *
import pickle
import os

In [12]:
data_path = Path("/home/mrdbarros/projetos/data/gquest_data/")

In [13]:
#export
print('Reading data...')
train_data = pd.read_csv(data_path/'train/train.csv')
test_data = pd.read_csv(data_path/'test/test.csv')
sample_submission = pd.read_csv(str(data_path/'sample_submission.csv'))
print('Reading data completed')

Reading data...
Reading data completed


In [14]:
train_data.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfa

In [15]:
 

with open(data_path/'vocab.pkl', 'rb') as vocab_file:
 
    # Step 3
    lm_vocab = pickle.load(vocab_file)
 
    # After config_dictionary is read from file
lm_vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 '/',
 'the',
 ',',
 '-',
 '.',
 'to',
 '\n▁',
 'a',
 ':',
 ')',
 'i',
 'is',
 '\n\n',
 'and',
 'of',
 '"',
 '(',
 'in',
 '\n',
 'it',
 'that',
 '\\',
 'you',
 '$',
 'for',
 ';',
 'this',
 '?',
 '=',
 'users',
 'https',
 '}',
 "'",
 'with',
 'on',
 'be',
 '3',
 '1',
 'if',
 '>',
 'have',
 'as',
 'not',
 'can',
 'are',
 '<',
 '2',
 '{',
 'or',
 'but',
 'do',
 '\n\n\n',
 '4',
 'http',
 'from',
 '5',
 'my',
 "'s",
 "n't",
 '10',
 '6',
 'an',
 ']',
 '7',
 '8',
 'questions',
 '9',
 '\n\n▁',
 'so',
 'how',
 'what',
 'at',
 'your',
 'there',
 'will',
 'would',
 'stackoverflow.com',
 'by',
 'use',
 'one',
 'when',
 'which',
 'like',
 'all',
 '0',
 'some',
 'does',
 '&',
 'get',
 'any',
 'using',
 'they',
 'then',
 '[',
 'was',
 '*',
 'just',
 'more',
 'should',
 'has',
 '+',
 'we',
 'other',
 'want',
 '#',
 "'m",
 'need',
 'only',
 'way',
 'no',
 'about',
 'am',
 'up',
 'time',
 'also',
 'new',
 'out',
 '

In [16]:
text_columns=['question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host']

In [16]:
train_data

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes instead of a macro lens?,"After playing around with macro photography on-the-cheap (read: reversed lens, rev. lens mounted on a straight lens, passive extension tubes), I would like to get further with this. The problems with the techniques I used is that focus is manual and aperture control is problematic at best. This limited my setup to still subjects (read: dead insects) Now, as spring is approaching, I want to be able to shoot live insects. I believe that for this, autofocus and settable aperture will be of great help.\n\nSo, one obvious but expensive option is a macro lens (say, EF 100mm Macro) However, I am ...",ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skinny.\n\n\n ...what am I losing when using tubes...?\n\n\nA very considerable amount of light! Increasing that distance from the end of the lens to the sensor can cut your light several stops. Combined with the fact that you'll usually shoot stopped down - expect to need to increase your ISO considerably.\n\nThe fact the macro's are usually considered very very sharp, although I believe that 70-200mm 2.8 is supposed to be quite sharp.\n\nThe ultra low distortion typical of many macros.\n\nI wouldn't worry too much about the bokeh since the DOF...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/what-am-i-losing-when-using-extension-tubes-instead-of-a-macro-lens,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a sprawl/metroplex... between downtown and a commercial district?,"I am trying to understand what kinds of places the spam values on p 231 refer to in the 5th Edition main book for Shadowrun.\n\nPer p 15, a sprawl is a plex, a plex is a ""metropolitan complex, short for metroplex"". Per Google a metroplex is "" a very large metropolitan area, especially one that is an aggregation of two or more cities"". A city downtown and sprawl downtown would tend to have similar densities, but for some reason the sprawl (which includes suburbs?) has a higher spam zone noise rating (p 231). Similarly, I'd think of a downtown as being more dense and noisy (e.g. Office bui...",russellpierce,https://rpg.stackexchange.com/users/8774,"It might be helpful to look into the definition of spam zone:\n\n(p.216) spam zone: An area flooded with invasive and/or viral AR advertising, causing noise.\n\nBecause a metroplex has so many marketing targets, it seems a safe assumption that marketers would drown the plex with spam. Spam from the less dense areas would bleed into the urban cores. A smaller city with less urban/suburban territory surrounding it ostensibly wouldn't have as much spam.\n",Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/what-is-the-distinction-between-a-city-and-a-sprawl-metroplex-between-downtow,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.000000,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole component pins,"I'm working on a PCB that has through-hole components on both sides of the board. The ""top"" side of the board is mounted flush to a Delrin plastic block (the only top-side component is a gas sensor that is fed air samples through hose fittings in the plastic block).\n\nThe flush mounting means that I have to add grooves to the plastic block to accommodate the soldered pins of the bottom-side components. Assuming a standard 0.062"" thickness FR4 board, how deep do I need to make the 

In [19]:
splits = RandomSplitter()(train_data)

In [22]:
df_tokenized,token_count=tokenize_df(train_data,text_columns)
x_tfms = [attrgetter("text"), Numericalize(vocab=lm_vocab)]
dsrc = DataSource(df_tokenized, splits=splits, tfms=[x_tfms, [attrgetter("question_asker_intent_understanding"), ToTensor()]], dl_type=SortedDL)

In [23]:
dbunch = dsrc.databunch(before_batch=pad_input, after_batch=Cuda)

In [25]:
dbunch.show_batch(max_n=2,trunc_at=60)

AttributeError: 'Tensor' object has no attribute 'show'